GC Net Data

In [2]:
!pip install pyarrow

You should consider upgrading via the '/Users/nilsfulde/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [1]:
!pip install git+https://github.com/GEUS-PROMICE/pyNEAD.git

  Cloning https://github.com/GEUS-PROMICE/pyNEAD.git to c:\users\mabj16ac\appdata\local\temp\4\pip-req-build-1zu17gro
  Resolved https://github.com/GEUS-PROMICE/pyNEAD.git to commit d811220c2d048d84d4612b045880d6ca914c39a4
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for nead: filename=nead-0.0.0-py2.py3-none-any.whl size=10663 sha256=4736ab4fc8e3fe62dc67c2c15b6c58ff60aed7a9afae1d2942330122de5b691b
  Stored in directory: C:\Users\mabj16ac\AppData\Local\Temp\4\pip-ephem-wheel-cache-zymeu2hk\wheels\7e\b5\9c\bf1fb23ef32c0fd131d467e2ee10e1bc483361a4dba16b3600
Successfully built nead


  Running command git clone --filter=blob:none --quiet https://github.com/GEUS-PROMICE/pyNEAD.git 'C:\Users\mabj16ac\AppData\Local\Temp\4\pip-req-build-1zu17gro'


In [19]:
import os
print(os.getcwd())
os.chdir('c:\\Users\\mabj16ac\\Desktop\\Thesis\\GEUS-Master-Thesis\\scripts')
os.chdir("../")

c:\Users\mabj16ac\Desktop\Thesis\GEUS-Master-Thesis\scripts


In [20]:
#!/usr/bin/env bash

# Latest L1 data: https://github.com/GEUS-Glaciology-and-Climate/GC-Net-level-1-data-processing/tree/main/L1
# API contents of latest L1 data (raw URLs etc.): https://api.github.com/repositories/319306521/contents/L1

import os
os.getcwd()
#os.chdir('C:\\Users\\nifu18ab\\Desktop\\GEUS-Master-Thesis')
os.chdir('c:\\Users\\mabj16ac\\Desktop\\Thesis\\GEUS-Master-Thesis\\scripts')
os.chdir("../")
print(os.getcwd())


try:
    os.mkdir("data")
    os.mkdir("data/gc_net/raw_data/data_daily")
    os.mkdir("data/gc_net/raw_data/data_hourly")
except:
    print('Overwritting existing data in "/data"')

import urllib.request

# Download data
print("Downloading daily data...\r")


# xargs -n 1 curl --silent -O --output-dir data_daily < ../metadata/urls_1.txt
for url in open("metadata/urls_1.txt"):
    # Split on the rightmost / and take everything on the right side of that
    name = url.rsplit("/", 1)[-1].replace("\r", "")
    # Strip /n at the end of filename
    name = name.strip()
    # Combine the name and the downloads directory to get the local filename
    filename = os.path.join("data/gc_net/raw_data/data_daily", name)

    # Download the file if it does not exist
    if not os.path.isfile(filename):
        urllib.request.urlretrieve(url, filename)


c:\Users\mabj16ac\Desktop\Thesis\GEUS-Master-Thesis
Overwritting existing data in "/data"


In [21]:
print("Downloading hourly data...\r")
# xargs -n 1 curl --silent -O --output-dir data_hourly < ../metadata/urls_2.txt
for url in open("metadata/urls_2.txt"):
    # Split on the rightmost / and take everything on the right side of that
    name = url.rsplit("/", 1)[-1].replace("\r", "")
    # Strip /n at the end of filename
    name = name.strip()
    # Combine the name and the downloads directory to get the local filename
    filename = os.path.join("data/gc_net/raw_data/data_hourly", name)

    # Download the file if it does not exist
    if not os.path.isfile(filename):
        urllib.request.urlretrieve(url, filename)

In [33]:
import pandas as pd
import nead

def process_gcnet_daily():
    print('PROCESSING GC_NET DAILY...')
    # Convert NEAD files to Pandas dataframes
    station = pd.read_csv("metadata/station_info.csv", header=0)
    dfs_daily = []

    for name, ID in zip(station.Name, station.ID):
        format_name = name.replace(" ", "")
        files = "data/gc_net/raw_data/data_daily/" + str(ID).zfill(2) + "-" + format_name + "_daily.csv"
        ds_daily = nead.read(files, index_col=0)
        df_daily = ds_daily.to_dataframe()
        df_daily.insert(
            loc=0, column="station_name", value=name
        )  # Add station_name column to each dataframe
        dfs_daily.append(df_daily)

    # Concatenate dataframes
    df_daily = pd.concat(dfs_daily).sort_index()

    # Delete irrelevant columns from dataframe (i.e. null columns and flag columns)
    # null_columns = df_daily.columns[df_daily.isnull().all()]
    # flag_columns = df_daily.filter(regex="flag$").columns
    # print(null_columns)
    # print(flag_columns)

    df_daily = df_daily.drop(
        columns=[
            "OSWR_max",
            "HW2_adj_flag",
            "P_adj_flag",
            "HW1_adj_flag",
            "OSWR_adj_flag",
            "HS1_adj_flag",
            "HS2_adj_flag",
            "TA3_adj_flag",
            "TA4_adj_flag",
            "DW1_adj_flag",
        ]
    )
    # Add season column to dataframe
    seasons = {
        1: "Winter",
        2: "Winter",
        3: "Spring",
        4: "Spring",
        5: "Spring",
        6: "Summer",
        7: "Summer",
        8: "Summer",
        9: "Autumn",
        10: "Autumn",
        11: "Autumn",
        12: "Winter",
    }

    # Extract the month from the index and use the dictionary to map it to the corresponding season
    df_daily["season"] = df_daily.index.month.map(seasons)

    # Add year column to dataframe
    df_daily["year"] = df_daily.index.strftime("%Y")

    # Add month column to dataframe
    df_daily["month"] = df_daily.index.strftime("%B")

    # Rename Index Column to Datetime
    df_daily = df_daily.reset_index(inplace=False)
    df_daily = df_daily.rename(columns={'timestamp': 'Datetime'}, inplace=False) 

    # Rename 'station_name' to file
    df_daily = df_daily.rename(columns={'station_name': 'file'}, inplace=False)

    # Add Day of Year & Day of Century 
    df_daily['DayOfYear'] = df_daily['Datetime'].dt.dayofyear 
    df_daily['DayOfCentury'] = df_daily['Datetime'].dt.dayofyear+365*(df_daily['Datetime'].dt.year-1)


    # # Add day column to dataframe
    # df_daily["day"] = df_daily.index.strftime("%d")

    # Add hour column to dataframe
    # df_daily["hour"] = df_daily.index.strftime("%h")

    # Change column headers
    #header = pd.read_csv('/content/drive/MyDrive/Master_Thesis/metadata/Masterdata_GCNET.csv', sep = ";")
    header = pd.read_csv('metadata/Masterdata_GCNET.csv', sep = ";")
    df_daily = df_daily.rename(columns = header.set_index('fields')['display_description'])

    print('SAVING PROCESSED GC_NET_DAILY TO CSVs...')
    counter = 1
    for station in df_daily['file'].unique():
        station_csv = df_daily.loc[df_daily['file'] == station]
        station_csv.to_csv('data/gc_net/daily_data/{num}_{station}.csv'.format(num=counter,station=station))
        counter += 1 
    print('FINISHED SAVING GC_NET DAILY TO CSVs')
    return df_daily

def process_gcnet_hourly():
    print('PROCESSING GC_NET HOURLY...')
    station = pd.read_csv("metadata/station_info.csv", header=0)
    dfs_hourly = []

    for name, ID in zip(station.Name, station.ID):
        format_name = name.replace(" ", "")
        files = "data/gc_net/raw_data/data_hourly/" + str(ID).zfill(2) + "-" + format_name + ".csv"
        ds_hourly = nead.read(files, index_col=0)
        df_hourly = ds_hourly.to_dataframe()
        df_hourly.insert(
            loc=0, column="station_name", value=name
        )  # Add station_name column to each dataframe
        dfs_hourly.append(df_hourly)

    # Concatenate dataframes
    df_hourly = pd.concat(dfs_hourly).sort_index()

    # Delete irrelevant columns from dataframe (i.e. null columns and flag columns)
    # null_columns = df_hourly.columns[df_hourly.isnull().all()]
    # flag_columns = df_hourly.filter(regex="flag$").columns
    # print(null_columns)
    # print(flag_columns)

    df_hourly = df_hourly.drop(
        columns=[
            "OSWR_max",
            "HW2_adj_flag",
            "P_adj_flag",
            "HW1_adj_flag",
            "OSWR_adj_flag",
            "HS1_adj_flag",
            "HS2_adj_flag",
            "TA3_adj_flag",
            "TA4_adj_flag",
            "DW1_adj_flag",
        ]
    )
    # Add year column to dataframe
    df_hourly["year"] = df_hourly.index.strftime("%Y")

    # Add month column to dataframe
    df_hourly["month"] = df_hourly.index.strftime("%B")

    # Add season column to dataframe
    seasons = {
        1: "Winter",
        2: "Winter",
        3: "Spring",
        4: "Spring",
        5: "Spring",
        6: "Summer",
        7: "Summer",
        8: "Summer",
        9: "Autumn",
        10: "Autumn",
        11: "Autumn",
        12: "Winter",
    }

    # Extract the month from the index and use the dictionary to map it to the corresponding season
    df_hourly["season"] = df_hourly.index.month.map(seasons)


    #Rename Index Column to Datetime
    df_hourly = df_hourly.reset_index(inplace=False)
    df_hourly = df_hourly.rename(columns={'timestamp': 'Datetime'}, inplace=False) 

    # Rename 'station_name' to file
    df_hourly = df_hourly.rename(columns={'station_name': 'file'}, inplace=False)

    # Add Day of Year & Day of Century 
    df_hourly['DayOfYear'] = df_hourly['Datetime'].dt.dayofyear 
    df_hourly['DayOfCentury'] = df_hourly['Datetime'].dt.dayofyear+365*(df_hourly['Datetime'].dt.year-1)


    # # Add day column to dataframe
    # df_hourly["day"] = df_hourly.index.strftime("%d")

    # # Add hour column to dataframe
    # df_hourly["hour"] = df_hourly.index.strftime("%h")


    #header = pd.read_csv('/content/drive/MyDrive/Master_Thesis/metadata/Masterdata_GCNET.csv', sep = ";")
    header = pd.read_csv('metadata/Masterdata_GCNET.csv', sep = ";")
    df_hourly = df_hourly.rename(columns = header.set_index('fields')['display_description'])

    print('SAVING PROCESSED GC_NET HOURLY TO CSVs')
    counter = 1
    for station in df_hourly['file'].unique():
        station_csv = df_hourly.loc[df_hourly['file'] == station]
        station_csv.to_csv('data/gc_net/hourly_data/{num}_{station}.csv'.format(num=counter,station=station))
        counter += 1 
    print('FINISHED SAVING GC_NET HOURLY TO CSVs')
    

    # Save dataframe as parquet file
    #df_hourly.to_parquet("data/df_hourly.gzip", compression="gzip", engine='pyarrow')
    

In [32]:
process_gcnet_daily()
process_gcnet_hourly()


,Datetime,file,shortwave_incoming_radiation,shortwave_outgoing_radiation,net_radiation,air_temperature_1,air_temperature_1_max,air_temperature_1_min,air_temperature_cs500_air1,air_temperature_cs500_air2,...,incoming_uv_radiation,incoming_longwave_radiation,surface_temperature_1,surface_temperature_2,net_radiation_maximum,year,month,season,DayOfYear,DayOfCentury
0,1990-06-01 01:00:00,Swiss Camp 10m,NaN,NaN,NaN,1.72,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1990,June,Summer,152,726137
1,1990-06-01 02:00:00,Swiss Camp 10m,NaN,NaN,NaN,1.42,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1990,June,Summer,152,726137
2,1990-06-01 03:00:00,Swiss Camp 10m,NaN,NaN,NaN,0.88,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1990,June,Summer,152,726137
3,1990-06-01 04:00:00,Swiss Camp 10m,NaN,NaN,NaN,-0.13,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1990,June,Summer,152,726137
4,1990-06-01 05:00:00,Swiss Camp 10m,NaN,NaN,NaN,-1.00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1990,June,Summer,152,726137
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3967230,2022-10-16 22:00:00,Humboldt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2022,October,Autumn,289,737954
3967231,2022-10-16 23:00:00,Humboldt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2022,October,Autumn,289,737954
3967232,2022-10-17 00:00:00,Humboldt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2022,October,Autumn,290,737955
3967233,2022-10-17 01:00:00,Humboldt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2022,October,Autumn,290,737955


New Promice

In [5]:
import pandas as pd
import numpy as np
import os

def process_hourly_data(csv_directory="..\..\PROMICE-AWS-toolbox\out\L4", add_meta_data=False):
        # List all CSV files in the directory
    csv_files = [f for f in os.listdir(csv_directory) if f.endswith('.csv')]

    # Combine all CSV files into a single DataFrame
    dfs = []
    for f in csv_files:
        df = pd.read_csv(os.path.join(csv_directory, f), index_col=False)
        df.insert(0, 'stid', f[:-7])
        dfs.append(df)
    df_hourly = pd.concat(dfs)

    #read metadata from promice repository
    station = pd.read_csv('../../PROMICE-AWS-toolbox/metadata/AWS_station_locations.csv', index_col=False)
    station.to_csv('../data/promice/new_promice/AWS_station_locations.csv', index=False)


    #display(output station and columns summary)
    print('Stations loaded:')
    display(df_hourly['stid'].unique())
    print('columns in dataset:')
    print(list(df_hourly.columns))
    print("FINISHED LOADING CSV's")

    # Add year column to dataframe
    df_hourly["Datetime"] = pd.to_datetime(df_hourly.time)

    #Rename Index Column to Datetime
    df_hourly = df_hourly.reset_index(inplace=False)

    if add_meta_data==True:
        add_nice_to_haves(df_hourly, 'hourly')

    header = pd.read_csv('../metadata/promice_header.csv', sep = ";")
    df_hourly = df_hourly.rename(columns = header.set_index('standard_name')['long_name'])
    print(df_hourly.columns)

    counter = 1
    for station in df_hourly['stid'].unique():
        station_csv = df_hourly.loc[df_hourly['stid'] == station]
        station_csv.to_csv('../data/promice/new_promice/hourly_data/{num}_{station}.csv'.format(num=counter,station=station))
        counter += 1 

    print("FINISHED PROCESSING HOURLY DATA")
    return df_hourly

def process_daily_data(dataframe=None, directory="../data/promice/new_promice/hourly_data", add_meta_data=False):
    
    if not isinstance(dataframe, pd.DataFrame):
        # List all CSV files in the directory
        csv_files = [f for f in os.listdir(directory) if f.endswith('.csv')]

        # Combine all CSV files into a single DataFrame
        dfs = []
        for f in csv_files:
            df = pd.read_csv(os.path.join(directory, f), index_col=False)
            dfs.append(df)
            df = pd.concat(dfs)
        print('finished loading csvs')
    else:
        df = dataframe.copy()

    # Define the date column that you want to group by (replace "date_column" with the name of your column)
    min_values_per_day = 20

    # Group the data by weather station and date
    grouped = df.groupby(['stid','date'])
    

    # Specify columns containing numerical values to be averaged
    columns_to_average = (df
                .select_dtypes(exclude=['object'])
                .drop(columns=['index', 'DayOfYear', 'DayOfCentury'])
                .columns
    )
    # Calculate the number of non-NaN values for each variable within each group
    valid_date_observations = grouped[columns_to_average].apply(lambda x: x.notna().sum() <= min_values_per_day)
    filter_excluded_columns = ['albedo' ]
    valid_date_observations[filter_excluded_columns] = False


    # Calculate average per day per station
    df_filtered = grouped[columns_to_average].mean()

    #Remove means with less than 20 observations per day
    df_masked = df_filtered.mask(valid_date_observations, np.nan)

    df_daily = df_masked.reset_index().copy()

    df_daily["Datetime"] = pd.to_datetime(df_daily['date'])

    if add_meta_data==True:
        df_daily = add_nice_to_haves(df_daily, 'daily')

    counter = 1
    for station in df_daily['stid'].unique():
        station_csv = df_daily.loc[df_daily['stid'] == station]
        station_csv.to_csv('../data/promice/new_promice/daily_data/{num}_{station}.csv'.format(num=counter,station=station))
        counter += 1 

    print("FINISHED PROCESSING HOURLY DATA")
    return df_daily

###########################################################################################
### Helper functions

def add_station_info(df):
    station = pd.read_csv('../data/promice/new_promice/AWS_station_locations.csv')
    station.rename(columns={'timestamp':'station_location_timestamp'})
    df.merge(station, how='left',on=['stid','stid'])
    return df

# helper functions for adding metadata like season, day, month, year, date, DayOfYear and DayOfCentury
def add_nice_to_haves(df, period):
    def add_season(df):
        seasons = {
            1: "Winter",
            2: "Winter",
            3: "Spring",
            4: "Spring",
            5: "Spring",
            6: "Summer",
            7: "Summer",
            8: "Summer",
            9: "Autumn",
            10: "Autumn",
            11: "Autumn",
            12: "Winter",
        }

        # Extract the month from the index and use the dictionary to map it to the corresponding season
        df["season"] = df['Datetime'].dt.month.map(seasons)
        return df

    def add_common(df):
        df["year"] = df['Datetime'].dt.strftime("%Y")
        df["month"] = df['Datetime'].dt.strftime("%B")
        df["date"] = df['Datetime'].dt.date
        df['DayOfYear'] = df['Datetime'].dt.dayofyear 
        df['DayOfCentury'] = df['Datetime'].dt.dayofyear+365*(df['Datetime'].dt.year-1)
        df = add_season(df)
        return df
        
    def add_day(df):
        # Add day column to dataframe
        df["day"] = df['Datetime'].dt.strftime("%d")
        return df

    def add_hour(df):
        # Add hour column to dataframe
        df["hour"] = df['Datetime'].dt.strftime("%h")
        return df

    if period == 'hourly':
        df = add_hour(
            add_day(
            add_common(df)))

    elif period == 'daily':
        df = add_day(
            add_common(df))
    elif period == 'monthly':
        df = add_common(df)
    
    return df 


In [6]:
os.chdir('c:\\Users\\mabj16ac\\Desktop\\Thesis\\GEUS-Master-Thesis\\scripts')

#header = pd.read_csv('../metadata/promice_header.csv', sep = ";")
#df_hourly = df_hourly.rename(columns = header.set_index('standard_name')['long_name'])
#df_hourly = process_hourly_data(add_meta_data=True)
#df_hourly = df_hourly.rename(columns = header.set_index('standard_name')['long_name'])

#df_hourly = process_hourly_data(directory="../data/new_promice/all_promice_data.parquet.gzip")
############################################

df_daily = process_daily_data(add_meta_data=True)
#df_daily = df_daily.rename(columns = header.set_index('standard_name')['long_name'])

#df_daily = process_daily_data(directory="../data/new_promice/all_promice_data_hourly.parquet.gzip", add_meta_data=True)

############################################
#df_monthly = process_monthly_data(dataframe=df_daily, add_meta_data=True)
#df_monthly = df_monthly.rename(columns = header.set_index('standard_name')['long_name'])

#df_monthly = process_monthly_data(directory='..\\Data\\new_promice\\all_promice_data_daily.parquet.gzip', add_meta_data=True)

### helper function to add station information such as # of booms, location and classification

# df_hourly = add_station_info(df_hourly)
# df_daily = add_station_info(df_daily)
# df_monthly = add_station_info(df_monthly)

finished loading csvs
FINISHED PROCESSING HOURLY DATA


In [ ]:
def promice_pull(directory='/data/promice/new_promice/daily_data'):
    csv_files = [f for f in os.listdir(directory) if f.endswith('.csv')]

    # Combine all CSV files into a single DataFrame
    dfs = []
    for f in csv_files:
        df = pd.read_csv(os.path.join(directory, f), index_col=False)
        df.insert(0, 'stid', f[:-7])
        dfs.append(df)
    df = pd.concat(dfs)
    return df

In [ ]:
C:\Users\mabj16ac\Desktop\Thesis\GEUS-Master-Thesis\data\promice\new_promice\hourly_data
null_columns = df_hourly.columns[df_hourly.isnull().all()]
flag_columns = df_hourly.filter(regex="flag$").columns
print("null columns:\n", null_columns)
print("flag_columns:\n", flag_columns)

In [ ]:
df_monthly.info

Promice Data

In [ ]:
!wget -r -e robots=off -nH --cut-dirs=3 --content-disposition "https://dataverse.geus.dk/api/datasets/:persistentId/dirindex?persistentId=doi:10.22008/FK2/8SS7EW"

In [9]:
#############################################
# Redundant

def process_monthly_data(dataframe=None, directory="", add_meta_data=False):

    if not isinstance(dataframe, pd.DataFrame):
        df = pd.read_parquet(directory)
    else:
        df = dataframe.copy()

    df['month_year'] = df['Datetime'].dt.to_period('M')
    # Define the date column that you want to group by (replace "date_column" with the name of your column)
    min_values_per_month = 24

    # Create a new column with the month and year of the date column

    # Group the data by weather station and date
    grouped = df.groupby(['stid','month_year'])

    # Specify columns containing numerical values to be averaged
    columns_to_average = (df
                .select_dtypes(exclude=['object'])
                .drop(columns=['Datetime', 'DayOfYear', 'DayOfCentury'])
                .columns
    )

    # Calculate the number of non-NaN values for each variable within each group
    valid_date_observations = grouped[columns_to_average].apply(lambda x: x.notna().sum() <= min_values_per_month)

    # Calculate average per day per station
    df_filtered = grouped[columns_to_average].mean()

    #Remove means with less than 20 observations per day
    df_masked = df_filtered.mask(valid_date_observations, np.nan)

    df_monthly = df_masked.reset_index().copy()

    df_monthly["Datetime"] = pd.to_datetime(df_monthly['month_year'].astype(str) + '-01')

    if add_meta_data==True:
        df_monthly = add_nice_to_haves(df_monthly, 'monthly')

    #display(df_monthly)

    # Convert the DataFrame to a compressed Parquet file
    output_file = "..\\Data\\new_promice\\all_promice_data_monthly.parquet.gzip"
    df_monthly.to_parquet(output_file, compression='gzip', engine='pyarrow')

    print('FINISHED PROCESSING MONTHLY DATA')
    return df_monthly

In [10]:
# save daily data 
# Get the files from the path provided in the OP
files = Path(path).glob('*_day_v03.txt')  # .rglob to get subdirectories

dfs = list()
for f in files:
    data = pd.read_csv(f, delimiter=r"\s+", engine='python')
    # .stem is method for pathlib objects to get the filename w/o the extension
    data['file'] = f.stem
    dfs.append(data)

df = pd.concat(dfs, ignore_index=True)

df = df.replace(-999, np.nan)

df["Month"] = df["MonthOfYear"]
df["Day"] = df["DayOfMonth"]

df["Datetime"] = pd.to_datetime(df[["Year", "Month", "Day"]], format='%Y%m%d')

# Save dataframe as parquet file
df.to_parquet("data/promice_daily.gzip", compression="gzip", engine='pyarrow')
#df.to_parquet("/content/drive/MyDrive/Master_Thesis/data/promice_daily.gzip", compression="gzip")

In [11]:
# save monthly data 
# Get the files from the path provided in the OP
files = Path(path).glob('*_month_v03.txt')  # .rglob to get subdirectories

dfs = list()
for f in files:
    data = pd.read_csv(f, delimiter=r"\s+", engine='python')
    # .stem is method for pathlib objects to get the filename w/o the extension
    data['file'] = f.stem
    dfs.append(data)

df = pd.concat(dfs, ignore_index=True)

df = df.replace(-999, np.nan)

df["Month"] = df["MonthOfYear"]

df["Datetime"] = pd.to_datetime(['{}-{}-01'.format(y, m) for y, m in zip(df.Year, df.Month)])

# Save dataframe as parquet file
df.to_parquet("data/promice_monthly.gzip", compression="gzip", engine='pyarrow')
#df.to_parquet("/content/drive/MyDrive/Master_Thesis/data/promice_monthly.gzip", compression="gzip")